In [2]:
import sys
sys.path.append("./src")
import pandas as pd
import os
import matplotlib.pyplot as plt
from IntegratedGradients import *
from model import *
import json
from tensorflow import keras
from util import *

In [3]:
with open("./config/age_parameters.json") as param:
    parameters = json.load(param)["load_data"]
param.close()

train_label = parameters["train_label_path"]
train_image = parameters["train_image_path"]
resize = parameters["resize"]
TRAIN_TEST_SPLIT = parameters["TRAIN_TEST_SPLIT"]

#model = OutputModel().assemble_full_model(resize, resize)
#model.load_model("../models/age_model_2/cp-0092.hdf5.ckpt")
model = keras.models.load_model("./models/age_model_2/cp-0092.hdf5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
model.output.shape[1]._value

9

In [4]:
#from IntegratedGradients import *
#model_param_path = "./models/age_model_2/cp-0092.hdf5"
ig = integrated_gradients(model)

Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.


In [ ]:
integrated_grad_pic(model_param_path, train_image, train_label, 2)

In [ ]:
def integrated_grad_pic(model_param_path, train_image_path, train_label_path, img_idx):
    model = keras.models.load_model(model_param_path)
    ig = integrated_gradients(model)
    
    dataset_dict = {
        'age_id': {
            0: '0-2', 
            1: '3-9', 
            2: '10-19', 
            3: '20-29', 
            4: '30-39',
            5: '40-49',
            6: '50-59',
            7: '60-69',
            8: "more than 70"
        }
    }
    img_name = "{}.jpg".format(img_idx)
    sample_path = os.path.join(train_image_path, img_name)
    sample_label_df = pd.read_csv(train_label_path)
    sample_label = sample_label_df[sample_label_df["file"].str.contains(img_name)]["age"].values[0]
    
    sample_image = preprocess_image(sample_path, resize, resize).reshape(1, 198, 198, 3)
    
    # Plot the true image.
    plt.figure(figsize = (5, 5))
    plt.imshow(sample_image.squeeze(), cmap="Greys")
    plt.xticks([],[])
    plt.yticks([],[])
    plt.title("Original image", fontsize=8)
    plt.savefig("./pictures/"+"Original_" + str(img_idx)+".png")

    # Generate explanation with respect to each of 10 output channels.
    exs = []
    output_prob = model.predict(sample_image).squeeze()
    for i in range(1,10):
        exs.append(ig.explain(sample_image.squeeze(), outc=i-1))
    exs = np.array(exs)

    # Plot them
    th = max(np.abs(np.min(exs)), np.abs(np.max(exs)))


    fig = plt.subplots(3,3,figsize=(15,15))
    for i in range(9):
        ex = exs[i]
        plt.subplot(3,3,i+1)
        plt.imshow(ex[:,:,0], cmap="seismic", vmin=-1*th, vmax=th)
        plt.xticks([],[])
        plt.yticks([],[])
        plt.title("heatmap for age range {} with probability {:.2f}".format(dataset_dict["age_id"][i],output_prob[i]), 
                  fontsize=10)
    plt.tight_layout()
    plt.savefig("./pictures/"+"integrated-viz_" + str(img_idx)+".png")
    plt.show()
    print("Ground Truth:", sample_label)
    print("Predicted:", dataset_dict["age_id"][np.argmax(output_prob)])
    
    
    